In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt

# Total Return Examples

Example 1: You bought TD bank shares for $\$72.50$ and sold one year later for $\$81.23$.  During that year, $2.75 of dividends were paid.  What is the total \% return?

Example 2: A 6\% $\$100$ face value bond was bought for $\$95.50$ and sold after one year for $\$94.45$. Assume that coupons are paid annually.  What is the total \% return?


In [2]:
#Variables for TD
Cash_Flow_TD=2.75
Begin_Value_TD=72.50
End_Value_TD=81.23

#Declare Variables for example 2 Bond
Cash_Flow_Bond= 6
Begin_Value_Bond=95.50
End_Value_Bond=94.45

#Calculate return percentage taking into acount beginning and end value, as well as any cash flows paid in the interim
Total_Return_TD= 100 * ((Cash_Flow_TD + (End_Value_TD - Begin_Value_TD))/Begin_Value_TD)
Total_Return_Bond= 100 * ((Cash_Flow_Bond + (End_Value_Bond - Begin_Value_Bond))/Begin_Value_Bond)
print("The Total Return for TD is: ", round(Total_Return_TD,2),"%", sep='')
print("The Total Return for the Bond is: ", round(Total_Return_Bond,2),"%", sep='')

The Total Return for TD is: 15.83%
The Total Return for the Bond is: 5.18%


## Percentage Returns, Expected Returns and Correlations


In [3]:
#To Learn about expected returns and correlations, let's look at telus and BCE.  
#Do you think they should be positively correlated?

Stock1='t.to'
Stock2='bce.to'
Ticker1 = yf.Ticker(Stock1)
Ticker2 = yf.Ticker(Stock2)

start_date = '2020-01-01'
end_date = '2022-04-25'

Stock1_hist = Ticker1.history(start=start_date, end=end_date)
Stock2_hist = Ticker2.history(start=start_date, end=end_date)
print(Ticker2.fast_info)
prices = pd.DataFrame(Stock1_hist['Close'])
prices.columns = [Stock1]
prices[Stock2] = Stock2_hist['Close']
#prices.head()

#Prior to coming to class - understand what we are doing here
#use pct_change() to convert daily prices to daily returns.
daily_returns = prices.pct_change()
daily_returns.drop(index=daily_returns.index[0], inplace=True)
#daily_returns.head()

#We want to go from daily returns to monthly returns.  Many ways to do this.  We could take an average return over
#the month for example.  Here we just want to get the overall monthly return, i.e., from the first day to the last day
#.resample('') will convert to monthly.  the parameter M will use the last day, MS will use the first day.  ffill fills
# in missing data (e.g., if we are using 'M' and there is no data on the last day, ffill will grab it from one day ahead)
monthly_returns=prices.resample('M').ffill().pct_change()
monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
#monthly_returns.head()

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']


##### Let's graph the monthly returns of each stock.  Do they tend to move together, apart, or no relationship?
plt.figure(figsize=(20,15))

plt.plot(monthly_returns.index,monthly_returns[Stock1], color='b', label=Stock1)
plt.plot(monthly_returns.index,monthly_returns[Stock2], color='r', label=Stock2)
plt.legend(loc='best')
plt.title('Monthly Returns')
plt.xlabel('Date')
plt.ylabel('Percent Change (%)')

plt.show()

### Expected returns

\begin{align*}
E(X)=\overline{X}=\frac{\sum x_i}{N}
\end{align*}

where $x_i$ are individual returns of some security $X$, $N$ is the total number of observations (time periods for us)

### Variance

\begin{align*}
\sigma^2_X=\frac{\sum(x_i-\overline{X})^2}{N}
\end{align*}

### Standard Deviation

\begin{align*}
\sigma_X=\sqrt{\frac{\sum(x_i-\overline{X})^2}{N}}
\end{align*}

### Covariance

Let's consider the relationship between two securities $X$ and $Y$.

\begin{align*}
COV(X,Y)=\frac{\sum(x_i-\overline{X})\times(y_i-\overline{Y})}{N}
\end{align*}

### Correlation

Very similar to Covariance, except standardized so that it takes a value between $-1$ and $1$

\begin{align*}
COR(X,Y)=\frac{COV(X,Y)}{\sigma_X \times \sigma_Y}
\end{align*}


In [4]:
print('Expected Monthly Returns:')
print(monthly_returns.mean())

print('')

print('Standard Deviations:')
print(monthly_returns.std())

print('')

print('Correlation:')
print(monthly_returns.corr())

Expected Monthly Returns:
t.to      0.013175
bce.to    0.010745
dtype: float64

Standard Deviations:
t.to      0.041882
bce.to    0.028864
dtype: float64

Correlation:
           t.to   bce.to
t.to    1.00000  0.55707
bce.to  0.55707  1.00000


## Beta

Consider Security $X$ with returns given by $x_i$, and let the correponding market returns be $r_M$.  Beta is then given by:

\begin{align*}
\beta=\frac{COV(x_i,r_M)}{\sigma^2(r_M)}
\end{align*}

In [7]:
Stock1='NFLX'
MarketIndex='^GSPC' #This is the symbol yfinance uses for the S&P 500
Ticker1 = yf.Ticker(Stock1)
Ticker2 = yf.Ticker(MarketIndex)

start_date = '2018-01-01'
end_date = '2022-05-25'

Stock1_hist = Ticker1.history(start=start_date, end=end_date)
MarketIndex_hist = Ticker2.history(start=start_date, end=end_date)


In [9]:
prices = pd.DataFrame(Stock1_hist['Close'])
prices.columns = [Stock1]
prices[MarketIndex] = MarketIndex_hist['Close']
print(prices.head())

#Do the same as before to get the monthly returns and then drop the first entry.

daily_returns = prices.pct_change()
daily_returns.drop(index=daily_returns.index[0], inplace=True)


print(daily_returns.head())

monthly_returns=prices.resample('M').ffill().pct_change()
monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
print(monthly_returns.head())

                                 NFLX        ^GSPC
Date                                              
2018-01-02 00:00:00-05:00  201.070007  2695.810059
2018-01-03 00:00:00-05:00  205.050003  2713.060059
2018-01-04 00:00:00-05:00  205.630005  2723.989990
2018-01-05 00:00:00-05:00  209.990005  2743.149902
2018-01-08 00:00:00-05:00  212.050003  2747.709961
                               NFLX     ^GSPC
Date                                         
2018-01-03 00:00:00-05:00  0.019794  0.006399
2018-01-04 00:00:00-05:00  0.002829  0.004029
2018-01-05 00:00:00-05:00  0.021203  0.007034
2018-01-08 00:00:00-05:00  0.009810  0.001662
2018-01-09 00:00:00-05:00 -0.012922  0.001303
                               NFLX     ^GSPC
Date                                         
2018-02-28 00:00:00-05:00  0.077987 -0.038947
2018-03-31 00:00:00-04:00  0.013625 -0.026884
2018-04-30 00:00:00-04:00  0.057931  0.002719
2018-05-31 00:00:00-04:00  0.125264  0.021608
2018-06-30 00:00:00-04:00  0.113282  0.004842

In [ ]:
#calculate the market variance (you will need to reference the column correponding to the market)
MarketVar=INSERT CALCULATION

#calculate beta by taking a covariance (.cov()), and using MarketrVar that was just defined.
Beta=INSERT CALCULATION

print(Beta)

print('')
print('Cell [0,0] is meaningless (to get beta, we divided by the variance of the market!)')
print('')

print('The Beta of Netflix is: ', Beta.iat[0,1])
print('The Beta of the Market is: ', Beta.iat[1,1])
